# EM 

Expectation Maximization是一种迭代算法,也有人称之为上帝算法,因为之后的很多算法都是以EM算法为基础的,比如HMM(隐马尔科夫),GMM(高斯混合),K-means等.

EM算法是一种迭代算法,每次迭代分为两步:E步,求期望(expectation);M步,求极大(maximization),所以这一算法被称为期望极大算法.

在理解EM算法之前,我们需要先知道什么是[极大似然估计](https://zh.wikipedia.org/wiki/%E6%9C%80%E5%A4%A7%E4%BC%BC%E7%84%B6%E4%BC%B0%E8%AE%A1)

### 1 极大似然估计

#### 问题描述:
    
现在有两个桶A,B;两个桶A,B都装有100个球,其中A桶有90个白球,10个黑球;B中有90个黑球,10个白球.现在随机从一个桶中拿出一个球是白球,那么这个白球是从A桶中拿出还是B桶中拿出.
    
很明显,在想这个问题的时候,我们直观上来理解的话,大概率下应该是从A桶中拿出来.那么这里的**"直观上来理解,大概率下"**就是用到了极大似然的思想了.因为我们认为来自A桶的概率大.

极大似然估计是建立在这样的思想上:已知某个参数能使这个样本出现的概率极大，我们当然不会再去选择其他小概率的样本，所以干脆就把这个参数作为估计的真实值.

那么转化成数学上的逻辑表示即为:

给定一个概率分布${\displaystyle D}$,已知其概率密度函数（连续分布）或概率质量函数（离散分布）为 ${\displaystyle f_{D}}$，以及一个分布参数 ${\displaystyle \theta }$,我们可以从这个分布中抽出一个具有 ${\displaystyle n}$个值的采样 ${\displaystyle X_{1},X_{2},\ldots ,X_{n}}$ ,利用 ${\displaystyle f_{D}}$计算出其似然函数:

${\displaystyle {\mbox{L}}(\theta \mid x_{1},\dots ,x_{n})=f_{\theta }(x_{1},\dots ,x_{n})=\prod p(x_i|\theta)} ,\theta \in \displaystyle{\Theta}$

在计算的时候我们需要取对数似然函数:

${ln \displaystyle {\mbox{L}}(\theta \mid x_{1},\dots ,x_{n})=ln f_{\theta }(x_{1},\dots ,x_{n})=ln\prod p(x_i|\theta)} ,\theta \in \displaystyle{\Theta}$

**注意:**
- 这里为什么要取对数?
    - 取对数的作用方便把product转化成累加的形式方便求解
    - 因为对数是Convex function.
        
- 怎么去极大似然?

    求$L(\theta)$对所有参数的偏导数,然后让这些偏导数为0,求解出来的就是极值点,但是需要注意的是这里求的极值点不一定是全局最优的点.

### 2 EM算法的引入

概率模型有时既有观测变量(observable variable),又含有隐变量(latent variable).如果概率模型的变量都是观测变量,那么给定数据,可以直接用极大似然估计法,或者贝叶斯估计模型参数.但是,当模型含有隐变量的时候就不能简单的使用极大似然估计.EM算法是含有隐变量的极大似然估计或极大后验概率估计.

#### 2-1 三硬币模型

我们先直观的来看一个例子,并完整的去分析这个例子,然后我们在GMM(Gaussian Mixture)中导出EM算法.

假设有3枚硬币,分别记作A,B,C.这些硬币正面的出现概率为$\pi,p,q.$进行如下实验:先掷硬币A,根据其结果选出硬币B或硬币C,正面选硬币B,反面选硬币C;然后掷选出的硬币,掷硬币的结果出现正面记作1,反面记作0;独立重复n次实验(这里n=10),观测结果如下:

$1,1,0,1,0,0,1,0,1,1$

假设只能观测到掷硬币的结果,不能观测到掷硬币的过程,问如何估计三硬币正面出现的概率,即三硬币模型的参数.

#### 2-2 阐述:

在这个例子中,我们发现没办法直接使用极大似然估计求出$\displaystyle {P(Y|\theta)}$,因为我们并不知道掷硬币A的结果.那么我们需要加入一个隐变量参数来得到整体的数据,因为只有整体完整的数据才能写出并计算出极大似然估计.

一般的,我们用$\displaystyle Y$来表示能够直接观测到的数据(**观测数据**),$\displaystyle Z$来表示看不到的数据(**隐变量**),在这里例子中体现为硬币A的掷结果.$\displaystyle Y$和$\displaystyle Z$连在一起为**完全数据(complete-data)**,观测数据$\displaystyle Y$又称为**不完全数据(incomplete-data)**.

假设给定观测数据$\displaystyle Y$,其概率分布是$\displaystyle {P(Y|\theta)}$,其中$\theta$是需要估计的模型参数,那么不完全数据$\displaystyle Y$的对数似然函数为$\displaystyle L(\theta)=logP(Y|\theta)$;j假设$\displaystyle {Y,Z}$的联合概率分布是$\displaystyle P(Y,Z|\theta)$,那么完全数据的对数似然函数是$log\displaystyle {P(Y,Z|\theta)}$,很明显$\displaystyle {P(Y|\theta)}$是$\displaystyle P(Y,Z|\theta)$的[边缘概率](https://zh.wikipedia.org/wiki/%E8%BE%B9%E7%BC%98%E5%88%86%E5%B8%83).

#### 2-3 求解:

在这个问题中,实验结果是可观测数据$Y = (y_1,y_2,...,y_N)$,硬币A的结果是不可观测数据$Z=(z_1,z_2,...,z_N)$且$Z$**只有两种**可能取值1和0.

1:正面,0:反面.

那么对于第$j$次实验:

$P(y_j|\theta) = \underset{z}{\sum}P(y_j,z|\theta)$
    
$=\underset{z}{\sum}P(z|\theta)P(y_j|z,\theta)$

$=P(z=1|\theta)P(y_j|z=1,\theta) + P(z=0|\theta)P(y_j|z=0,\theta)$

$= \left\{\begin{matrix}
\pi p+(1-\pi)q, &if y_j=1; \\ 
 \pi (1-p) + (1-\pi)(1-q),& if y_j=0.
\end{matrix}\right.$  ①

$=\pi p^{y_j}(1-p)^{1-y_j}+(1-\pi)q^{y_j}(1-q)^{1-y_j}$

所以有:

$P(Y|\theta)=\prod_{j=1}^{N}P(y_j|\theta)=\prod_{j=1}^{N}(\pi p^{y_j}(1-p)^{1-y_j}+(1-\pi)q^{y_j}(1-q)^{1-y_j})$

a. E-step,求期望(Q函数):

a.1: Q函数的定义:$Q(\theta,\theta^{(g)})=E_{z}[logP(Y,Z|\theta)|Y,\theta^{(g)}]=\underset{z} \sum P(Z|Y,\theta^{(g)})logP(Y,Z|\theta)$

$Q(\theta,\theta^{(g)})=\underset{z} \sum P(z|Y,\theta_{n})logP(Y,z|\theta)$

$=\sum_{j=1}^{N}\{\underset{z} \sum P(z|y_j,\theta_{n})lnP(y_j,z|\theta)\}$

$=\sum_{j=1}^{N}\{ P(z=1|y_j,\theta_n)lnP(y_j,z=1|\theta)+P(z=0|y_j,\theta)lnP(y_j,z=0|\theta)\} $

a.2:先求$P(z|y_j,\theta_n),$

$P(z=1|y,\theta)$ 

$=\frac{P(z=1,y_j|\theta_n)}{\underset{z}\sum P(z=1,y_j|\theta_n)}$ [Bayes](https://zh.wikipedia.org/wiki/%E8%B4%9D%E5%8F%B6%E6%96%AF%E5%AE%9A%E7%90%86)

$=\frac{P(y_j|z=1,\theta_n) P(z=1|\theta_n)}{\underset{z}\sum P(y_j|z=1,\theta_n) P(z=1|\theta_n)}$ 

$\Rightarrow $

$P(z|y_j,\theta_n)=\left\{\begin{matrix}
\frac{\pi_n p_{n}^{y_j} (1-p_n)^{1-y_j}}{\pi_n p^{y_j}_n(1-p_n)^{1-y_j}+(1-\pi_n)q_{n}^{y_j}(1-q_n)^{1-y_j}} =\mu_{j,n};& if z=1 &②\\ 
1-\mu_{j,n}& if z=0 & ③
\end{matrix}\right.
.$

a.3:再求$P(y_j,z|\theta)=P(z|\theta)P(y_j|z,\theta),$

根据①分析可得

$P(y_j,z|\theta)=\left\{\begin{matrix}
\pi p^{y_j}(1-p)^{1-y_j} & if z=1; \\ 
 (1-\pi)q^{y_j}(1-q)^{1-y_j}& if z=0;
\end{matrix}\right.$

a.4:因此$Q$函数的表达式为:

$Q(\theta|\theta_n)=\sum_{j=1}^{N}\{\mu_{j,n}ln[\pi p^{y_j}(1-p)^{1-y_j}] + (1-\mu_{j,n})ln[(1-\pi)q^{y_j}(1-q)^{1-y_j}]\}$

**Ps:**
上面②实际上是在求掷硬币A后的结果是正面也就是下次是掷硬币B的结果概率,那么就是掷硬币B的概率除上联合概率,③即是②的对立面.

b. M-step,求$Q$的极大值:

b.1 求$Q$函数对各个参数($\pi,p,q$)的偏导并等于0

### $\frac{\partial Q(\theta|\theta_n)} {\partial \pi} = \sum_{j=1}^{N}\{\frac{\mu_{j,n}ln[\pi p^{y_j}(1-p)^{1-y_j}] + (1-\mu_{j,n})ln[(1-\pi)q^{y_j}(1-q)^{1-y_j}]}{\partial \pi}\}$

### $= \sum_{j=1}^{N}\{\mu_{j,n}\frac{p^{y_j}(1-p)^{1-y_j}}{\pi p^{y_j}(1-p)^{1-y_j}}+(1-\mu_{j,n})\frac{-q^{y_j}(1-q)^{1-y_j}}{(1-\pi)q^{y_j}(1-q)^{1-y_j}}\}$

### $=\sum_{j=1}^{N}\{\frac{\mu_{j,n}-\pi }{\pi (1-\pi)}\}$

### $=\frac{(\sum_{j=1}^{N}\mu_{j,n})-n\pi}{\pi (1-\pi)} = 0$

### $\Rightarrow $

### $\pi_{n+1}=\frac{1}{n}\sum_{j=1}^{N}\mu_{j,n}$

### $\frac{\partial Q(\theta|\theta_n)} {\partial p} = \sum_{j=1}^{N}\{\frac{\mu_{j,n}ln[\pi p^{y_j}(1-p)^{1-y_j}] + (1-\mu_{j,n})ln[(1-\pi)q^{y_j}(1-q)^{1-y_j}]}{\partial p}\}$

### $=\sum_{j=1}^{N}\{\mu_{j,n}\frac{\pi (y_j p^{y_j -1}(1-p)^{1-y_j}+p^{y_j}(-1)(1-y_j)(1-p)^{1-y_j-1})}{\pi p^{y_j}(1-p)^{1-y_j}} + 0\}$

### $=\sum_{j=1}^{N}\{\frac{\mu_{j,n}(y_j-p)}{p(1-p)}\}$

### $=\frac{(\sum_{j=1}^{N} \mu_{j,n}y_j)-(p\sum_{j=1}^{N}\mu_{j,n})}{p(1-p)} = 0$

### $\Rightarrow \cdots$

### $p_{n+1}=\frac{\sum_{j=1}^{N}\mu_{j,n}y_j}{\sum_{j=1}^{N}\mu_{j,n}},q_{n+1}=\frac{\sum_{j=1}^{N}(1-\mu_{j,n})y_j}{\sum_{j=1}^{N}(1-\mu_{j,n})}$

当参数都已经计算出来了,就可以开始迭代了,我们发现在迭代参数中,我们需要先求得后验概率$P(z|y_j,\theta_n)$中的$\mu_{j,n}$,所以迭代应该从后验概率开始,先计算后验概率,得到后验概率后计算各个参数.

比如第一次迭代:

$\pi^{(0)}=0.5,p^{(0)}=0.5,q^{(0)}=0.5 \Rightarrow \mu^{(0)}=0.5$

得到

$\pi^{(1)}=0.5,p^{(1)}=0.6,q^{(1)}=0.6 \Rightarrow \mu^{(1)}=0.5$

.....

最后在一定阈值内达到最优值,但是这个最优值不一定是全局最优.在EM证明中会提到.

Ps:
- 对于$P(Y|\theta)$,理论上可以直接极大似然求出各个参数的估计值,但是由于边缘概率的存在,如果将其展开,会异常复杂,无法求解.
- 如果初始值选取的不同,那么迭代结果也会不同,所以EM很依赖于初始值.
- <p style="color:orange">实际上后验概率就是各个样本点属于不同混合子模型的概率值,将在(Gaussian Mixture)中深入理解</p>
- 隐变量的选择只有两种
- 由于EM是基于极大似然估计的,所以同极大似然估计一样,我们必须要知道数据服从的分布才能使用

### 3 EM算法

EM算法:

输入:观测变量数据Y,隐变量数据Z,联合分布$P(Y,Z|\theta)$,条件分布(后验概率)$P(Z|Y,\theta)$;
输出:模型参数$\theta$

(1) 选择初始参数进行迭代

(2) E步: 从后验概率开始计算出各个参数所需要的值.

(3) M步: 使得$Q(\theta,\theta^{g})$极大化$\theta$

$\theta^{(g+1)} = \underset{\theta} argmaxQ(\theta,\theta^{(g)})$

(4) 不断迭代(2),(3)直到收敛,收敛的条件是可以给予一个较小的阈值$\varepsilon$使得,

$||Q^{(g+1)}-Q^{(g)}||\leqslant \varepsilon$

### 4 Model

In [1]:
import numpy as np

In [2]:
def model(pi,p,q,iter_,is_print=False):
    """
    Implementation ME model -1.
    
    parameters:
    ----------
        pi: prob A
        p: prob B
        q: prob C
        iter_: number of iter.
        is_print: is print.
        
    Return:
    ------
        The best pi ,p ,q.
    
    """
    y = np.array([1,1,0,1,0,0,1,0,1,1]) # labels
    n = y.shape[0] # length of y
    
    for i in range(iter_):
        
        # E-step
        pro_1 = pi * np.power(p,y) * np.power((1-p),(1-y))
        pro_2 = (1 - pi) * np.power(q,y)* np.power((1-q),(1-y))
        mu = pro_1 / (pro_1 + pro_2)
        
        # M-step
        pi = np.sum(mu)/n
        p = np.sum(mu * y)/np.sum(mu)
        q = np.sum((1-mu) * y)/ np.sum(1-mu)
        
        if is_print:
            
            print('pi:{},p:{},q:{}'.format(pi,p,q))
            
    return pi,p,q


In [3]:
pi,p,q = 0.4,0.6,0.7
pi,p,q = model(pi,p,q,3)

print('best pi:{},best p:{},best q:{}'.format(pi,p,q))

best pi:0.40641711229946526,best p:0.5368421052631581,best q:0.6432432432432431


-----------